In [1]:
import pandas as pd
import json
import datetime

In [2]:
def format_date(date):

    formatted_date = '.'.join(str(date).split('-'))

    return formatted_date

In [15]:
today = datetime.date.today()
today = format_date(today)

In [16]:
from os import listdir

data_dir = "data"
files = listdir(data_dir)

latest_file = [f for f in files if f[-3:] == "csv"][-1]
latest_iter = latest_file[-5]

latest_df = pd.read_csv(data_dir + "\\" + latest_file)
latest_read_date = latest_df["date_readed"].max()


In [17]:
raw_data_path = r"data\flights_data.json"
output_csv_path = f"data\\flights_{int(latest_iter) + 1}.csv"

In [18]:
with open(raw_data_path) as f:
    data = json.load(f)["items"]

df = pd.DataFrame(data)
df["stops_num"] = df.stops.str["stops"]
max_stops = df["stops_num"].max()

df2 = pd.DataFrame(df.stops.str["stops_data"].tolist())

for i in range(max_stops):
    df[f"stop_{i+1}_duration"] = df2[i].str["duration"]
    df[f"stop_{i+1}_place"] = df2[i].str["place"]

df = df.drop('stops', axis=1)

In [19]:
df = df[df["date_readed"] > latest_read_date]

In [20]:
df[df["date_readed"] == today].sort_values("price").head(10)

,date_readed,departure_date,arrival_date,departure_time,arrival_time,return_date,price,airline,duration,origin,destination,stops_num,stop_1_duration,stop_1_place,stop_2_duration,stop_2_place
448,2022.11.10,"Friday, February 3 2023","Friday, February 3 2023",12:30 PM,7:00 PM,2023.02.08,561,"[Wizz Air, easyJet]",6 hr 30 min,Kraków,Barcelona,1,1 hr 55 min,Gatwick,None,None
428,2022.11.10,"Friday, February 3 2023","Friday, February 3 2023",12:30 PM,7:00 PM,2023.02.06,571,"[Wizz Air, easyJet]",6 hr 30 min,Kraków,Barcelona,1,1 hr 55 min,Gatwick,None,None
417,2022.11.10,"Thursday, February 2 2023","Thursday, February 2 2023",1:25 PM,5:35 PM,2023.02.08,596,[Lufthansa],4 hr 10 min,Kraków,Barcelona,1,50 min,Munich,None,None
414,2022.11.10,"Thursday, February 2 2023","Thursday, February 2 2023",7:30 AM,12:05 PM,2023.02.08,596,[Lufthansa],4 hr 35 min,Kraków,Barcelona,1,1 hr 15 min,Munich,None,None
444,2022.11.10,"Friday, February 3 2023","Friday, February 3 2023",1:25 PM,5:35 PM,2023.02.08,596,[Lufthansa],4 hr 10 min,Kraków,Barcelona,1,50 min,Munich,None,None
445,2022.11.10,"Friday, February 3 2023","Friday, February 3 2023",6:45 AM,9:30 AM,2023.02.08,608,[Wizz Air],2 hr 45 min,Kraków,Barcelona,0,None,None,None,None
391,2022.11.10,"Thursday, February 2 2023","Thursday, February 2 2023",6:00 AM,12:15 PM,2023.02.06,610,[KLM],6 hr 15 min,Kraków,Barcelona,1,2 hr 5 min,Amsterdam,None,None
415,2022.11.10,"Thursday, February 2 2023","Thursday, February 2 2023",6:00 AM,12:15 PM,2023.02.08,610,[KLM],6 hr 15 min,Kraków,Barcelona,1,2 hr 5 min,Amsterdam,None,None
429,2022.11.10,"Friday, February 3 2023","Saturday, February 4 2023",4:50 PM,10:25 AM,2023.02.06,610,[KLM],17 hr 35 min,Kraków,Barcelona,1,13 hr 30 min,Amsterdam,None,None
449,2022.11.10,"Friday, February 3 2023","Saturday, February 4 2023",4:50 PM,10:25 AM,2023.02.08,610,[KLM],17 hr 35 min,Kraków,Barcelona,1,13 hr 30 min,Amsterdam,None,None


In [9]:
import os
from azure.storage.blob import BlobServiceClient

connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

ModuleNotFoundError: No module named 'azure'

In [ ]:
container_name = "mol"
container_client= blob_service_client.get_container_client(container_name)

In [ ]:
df.to_csv(output_csv_path)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=output_csv_path)
print("\nUploading to Azure Storage as blob:\n\t" + output_csv_path)

with open(output_csv_path, "rb") as f:
    blob_client.upload_blob(f, overwrite=True)


Uploading to Azure Storage as blob:
	data\flights_3.csv
